In [1]:
#Exercise 5.7:
# The customer with customerID ALFKI has bought a series of products. Determine which other 
# customers have bought most of the same products (product types – 10 apples is no better than 1 apple).

import pymongo
from pymongo import MongoClient
db = MongoClient()['Northwind']  # db=MongoClient().Northwind

ConnectionFailure: [Errno 10061] No connection could be made because the target machine actively refused it

In [2]:
# Firstly, we need find product ordered by ALFKI by following this pipeline
# CustomerID -> OrderID -> ProductID
order_id=[]
for item in db.orders.find({"CustomerID":"ALFKI"}):
    order_id.append(item["OrderID"])
print order_id


NameError: name 'db' is not defined

In [7]:
product_id=[]
for item in db['order-details'].find({"OrderID":{"$in":order_id}}):
    product_id.append(item["ProductID"])
print product_id,len(product_id)

# unify ProductID avoiding repeatition 
from collections import Counter
uni_product_id=[k for k,v in Counter(product_id).items() if v>0]
print uni_product_id,len(uni_product_id)


[28, 39, 46, 63, 3, 76, 59, 77, 6, 28, 58, 71] 12
[3, 6, 39, 71, 76, 77, 46, 58, 59, 28, 63] 11


In [16]:
# in order to find which other customers ordered the same product, firstly we need to find 
# which other orders have ordered the same products
o_id=[]
for item in db['order-details'].find({"ProductID":{"$in":uni_product_id}}):
    for j in range(len(order_id)):
        if item["OrderID"]!= order_id[j]:
            o_id.append(item["OrderID"])
            #print item["OrderID"], item["ProductID"]
#print o_id,len(o_id)


from collections import Counter
uni_o_id=[k for k,v in Counter(o_id).items() if v>0] # V>0: find DISTINCT numbers in a list
print uni_o_id,len(uni_o_id)   # 274 other orders has ordered the same products
#Counter(o_id).most_common()


[10757, 10762, 10839, 10764, 10253, 10255, 10256, 10257, 10771, 10772, 10849, 11012, 10267, 10780, 10784, 10273, 10274, 10275, 10277, 10278, 10929, 10287, 10289, 10290, 10803, 10804, 10293, 11017, 10808, 10297, 10298, 10879, 10812, 10813, 10302, 10304, 10305, 10309, 10823, 10312, 10314, 10827, 10318, 10319, 10320, 10323, 10324, 10325, 10327, 10328, 10841, 11031, 10333, 10847, 10768, 10851, 10341, 10343, 10857, 10347, 10860, 10863, 10865, 10871, 10360, 10361, 10363, 10364, 10366, 10367, 10368, 10373, 10374, 10377, 10378, 10379, 10892, 10946, 10895, 10387, 10901, 10390, 10904, 10395, 10396, 10399, 10401, 10402, 10405, 10406, 10407, 10921, 10410, 10411, 10924, 10413, 10927, 10928, 10417, 10930, 10421, 10934, 10423, 10425, 10938, 11039, 10428, 10429, 10430, 10943, 10434, 10947, 10436, 10949, 10952, 10986, 10443, 10445, 10447, 10961, 10962, 10451, 10452, 10454, 10455, 10457, 10458, 10459, 10974, 10789, 10976, 10977, 10466, 10979, 10473, 10474, 10475, 10477, 10479, 10480, 10994, 10483, 10485

In [10]:
cus_id=[]
for item in db.orders.find({"OrderID":{"$in":uni_o_id}}):
    cus_id.append(item["CustomerID"])
    #print item["OrderID"], item["CustomerID"]
print cus_id,len(cus_id)

uniquecusID=[]
for i in cus_id:
    if i not in uniquecusID:
        uniquecusID.append(i)
print uniquecusID,len(uniquecusID)


[u'OTTIK', u'BLONP', u'HUNGO', u'RATTC', u'MORGK', u'TRAIH', u'VAFFE', u'QUEDE', u'BLAUS', u'MEREP', u'OLDWO', u'LEHMS', u'WHITC', u'RATTC', u'HILAA', u'VAFFE', u'FURIB', u'MEREP', u'SAVEA', u'BLAUS', u'GREAL', u'GREAL', u'MEREP', u'ISLAT', u'THECR', u'BERGS', u'KOENE', u'WARTH', u'WELLI', u'HUNGO', u'QUEDE', u'MAISD', u'BERGS', u'ALFKI', u'SAVEA', u'QUICK', u'ERNSH', u'BERGS', u'FRANK', u'FRANK', u'HUNGO', u'LONEP', u'BLONP', u'VAFFE', u'HANAR', u'ALFKI', u'QUICK', u'WHITC', u'LINOD', u'SAVEA', u'HUNGO', u'ALFKI', u'FOLKO', u'OLDWO', u'SAVEA', u'SAVEA', u'BONAP', u'QUEDE', u'REGGC', u'LETSS', u'BERGS', u'GOURL', u'WHITC', u'BOTTM', u'AROUT', u'QUICK', u'PICCO', u'ISLAT', u'WARTH', u'SAVEA', u'FOLKO', u'ERNSH', u'AROUT', u'BONAP', u'ERNSH', u'LEHMS', u'LILAS', u'MAGAA', u'FOLIG', u'KOENE', u'WELLI', u'SEVES', u'OLDWO', u'REGGC', u'RICAR', u'LILAS', u'BONAP', u'TRADH', u'ALFKI', u'BERGS', u'TRADH', u'LINOD', u'SUPRD', u'QUICK', u'SAVEA', u'KOENE', u'RICAR', u'BERGS', u'FRANR', u'HILAA',

In [17]:
cus_id=[]
o_id=[]
p_id=[]
for item in db['order-details'].find({"ProductID":{"$in":uni_product_id}}):
    #for j in range(len(order_id)):
        #if item["OrderID"]!= order_id[j]:
    for k in db.orders.find({"OrderID":item["OrderID"]}):
        cus_id.append(k["CustomerID"])   #customers who ordered the 11 products that ALFKI ordered as well
        o_id.append(k["OrderID"])        
        p_id.append(item["ProductID"])
        #dict = {'Name': 'Zara', 'Age': 7, 'Class': 'First'};
        dict ={'OrderID':o_id,'ProductID':p_id,'CustomerID': cus_id}
        print item["OrderID"], item["ProductID"], k["CustomerID"]
print cus_id,len(cus_id)

# 76 customers have bought most of the same products
uniquecusID=[]
for i in cus_id:
    if i not in uniquecusID:
        uniquecusID.append(i)
print uniquecusID,len(uniquecusID)

from collections import Counter
Counter(cus_id).most_common() 

10253 39 HANAR
10255 59 RICSU
10256 77 WELLI
10257 77 HILAA
10267 59 FRANK
10267 76 FRANK
10257 39 HILAA
10273 76 QUICK
10274 71 VINET
10275 59 MAGAA
10277 28 MORGK
10278 59 BERGS
10278 63 BERGS
10287 46 RICAR
10289 3 BSBEV
10290 77 COMMI
10293 63 TORTU
10297 39 BLONP
10298 59 HUNGO
10302 28 SUPRD
10304 59 TORTU
10304 71 TORTU
10305 39 OLDWO
10309 6 HUNGO
10309 71 HUNGO
10312 28 WANDK
10314 58 RATTC
10318 76 ISLAT
10319 28 TORTU
10319 76 TORTU
10320 71 WARTH
10323 39 KOENE
10324 46 SAVEA
10324 59 SAVEA
10324 63 SAVEA
10325 6 KOENE
10327 58 FOLKO
10328 59 FURIB
10333 71 WARTH
10341 59 SIMOB
10343 76 LEHMS
10347 39 FAMIA
10360 28 BLONP
10361 39 QUICK
10363 76 DRACD
10364 71 EASTC
10366 77 GALED
10367 77 VAFFE
10368 28 ERNSH
10373 58 HUNGO
10373 71 HUNGO
10374 58 WOLZA
10377 28 SEVES
10377 39 SEVES
10378 71 FOLKO
10379 63 QUEDE
10387 28 SANTG
10387 71 SANTG
10387 59 SANTG
10390 46 ERNSH
10395 46 HILAA
10396 71 FRANK
10399 76 VAFFE
10399 77 VAFFE
10399 71 VAFFE
10401 71 RATTC
10402 63 ERNS

[(u'ERNSH', 15),
 (u'HUNGO', 14),
 (u'SAVEA', 13),
 (u'QUICK', 12),
 (u'ALFKI', 12),
 (u'RATTC', 11),
 (u'VAFFE', 10),
 (u'BERGS', 10),
 (u'FOLKO', 9),
 (u'SUPRD', 9),
 (u'FRANK', 8),
 (u'HILAA', 8),
 (u'KOENE', 7),
 (u'TORTU', 6),
 (u'QUEDE', 6),
 (u'WARTH', 6),
 (u'LILAS', 6),
 (u'WHITC', 6),
 (u'QUEEN', 6),
 (u'BONAP', 6),
 (u'LEHMS', 5),
 (u'BLONP', 5),
 (u'ISLAT', 5),
 (u'LINOD', 5),
 (u'OTTIK', 4),
 (u'RICSU', 4),
 (u'RICAR', 4),
 (u'SANTG', 4),
 (u'MEREP', 4),
 (u'LAMAI', 4),
 (u'HANAR', 4),
 (u'REGGC', 4),
 (u'BOTTM', 4),
 (u'MORGK', 4),
 (u'OLDWO', 4),
 (u'GOURL', 3),
 (u'TRADH', 3),
 (u'WELLI', 3),
 (u'COMMI', 3),
 (u'WILMK', 3),
 (u'BSBEV', 3),
 (u'LONEP', 3),
 (u'BLAUS', 3),
 (u'SIMOB', 3),
 (u'OCEAN', 3),
 (u'MAISD', 3),
 (u'FURIB', 3),
 (u'SEVES', 3),
 (u'FOLIG', 2),
 (u'GREAL', 2),
 (u'DRACD', 2),
 (u'EASTC', 2),
 (u'CHOPS', 2),
 (u'MAGAA', 2),
 (u'PICCO', 2),
 (u'AROUT', 2),
 (u'TRAIH', 2),
 (u'FAMIA', 2),
 (u'GALED', 2),
 (u'PRINI', 1),
 (u'VINET', 1),
 (u'THECR', 1),


In [13]:
dict
dict.get('CustomerID')

[u'HANAR',
 u'HANAR',
 u'HANAR',
 u'HANAR',
 u'HANAR',
 u'HANAR',
 u'RICSU',
 u'RICSU',
 u'RICSU',
 u'RICSU',
 u'RICSU',
 u'RICSU',
 u'WELLI',
 u'WELLI',
 u'WELLI',
 u'WELLI',
 u'WELLI',
 u'WELLI',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'FRANK',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'HILAA',
 u'QUICK',
 u'QUICK',
 u'QUICK',
 u'QUICK',
 u'QUICK',
 u'QUICK',
 u'VINET',
 u'VINET',
 u'VINET',
 u'VINET',
 u'VINET',
 u'VINET',
 u'MAGAA',
 u'MAGAA',
 u'MAGAA',
 u'MAGAA',
 u'MAGAA',
 u'MAGAA',
 u'MORGK',
 u'MORGK',
 u'MORGK',
 u'MORGK',
 u'MORGK',
 u'MORGK',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'BERGS',
 u'RICAR',
 u'RICAR',
 u'RICAR',
 u'RICAR',
 u'RICAR',
 u'RICAR',
 u'BSBEV',
 u'BSBEV',
 u'BSBEV',
 u'BSBEV',
 u'BSBEV',
 u'BSBEV',
 u'COMMI',